In [1]:
initial_dna_sequence =list( "A9B6C9D6E9F6G9H6I9J6K")
forward_9_positions=[1,5,9,13,17]#computer positions
    
backward_6_positions=[3,7,11,15,19]#computer positions

In [2]:
# -*- coding: utf-8 -*-

class CompositeElement:
    def __init__(self, name):
        self.name = name
        self._composite = None
    
    def composite(self):
        return self._composite
    
    def set_composite(self, other):
        self._composite = other

class CompositeMapping:
    def __init__(self):
        self.elements = {}
    
    def add_pair(self, element1, element2):
        # Create CompositeElement instances if they don't exist
        if element1 not in self.elements:
            self.elements[element1] = CompositeElement(element1)
        if element2 not in self.elements:
            self.elements[element2] = CompositeElement(element2)
        
        # Set up the composite relationship
        self.elements[element1].set_composite(self.elements[element2])
        self.elements[element2].set_composite(self.elements[element1])
    
    def get_element(self, name):
        return self.elements.get(name)

def check_encoding():
    """Check if the file is properly encoded in UTF-8"""
    try:
        # Try to encode and decode a Greek character
        test_char = 'α'
        test_char.encode('utf-8').decode('utf-8')
        return True
    except UnicodeError:
        print("Warning: There might be encoding issues with Greek characters")
        return False

def safe_print(text):
    """Safely print text that might contain special characters"""
    try:
        print(text)
    except Exception as e:
        print(f"Error printing: {str(e)}")
        # Fallback to repr() for problematic characters
        print(repr(text))



def print_with_prime_coloring(text):
    """Print text with prime elements in blue color"""
    # ANSI color codes
    BLUE = '\033[94m'
    RESET = '\033[0m'
    
    if isinstance(text, list):
        # Handle list input
        for item in text:
            if isinstance(item, str):
                parts = item.split('_prime')
                if len(parts) > 1:
                    colored_text = parts[0] + BLUE + '_prime' + RESET + parts[1]
                    safe_print(colored_text)
                else:
                    safe_print(item)
            else:
                safe_print(item)
    else:
        # Handle string input
        parts = text.split('_prime')
        if len(parts) > 1:
            colored_text = parts[0] + BLUE + '_prime' + RESET + parts[1]
            safe_print(colored_text)
        else:
            safe_print(text)

def create_mapping():# Check encoding first
    if not check_encoding():
        print("Please ensure this file is saved with UTF-8 encoding")
        return

    # Create a composite mapping
    composite_mapping = CompositeMapping()
    
    # Define all the letter sets
    latin_upper = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    latin_lower = 'abcdefghijklmnopqrstuvwxyz'
    greek_upper = 'ΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩ'
    greek_lower = 'αβγδεζηθικλμνξοπρστυφχψω'
    
    # Define Latin names
    latin_names_upper = [
        'ALPHA', 'BETA', 'GAMMA', 'DELTA', 'EPSILON', 'ZETA', 'ETA', 'THETA',
        'IOTA', 'KAPPA', 'LAMBDA', 'MU', 'NU', 'XI', 'OMICRON', 'PI', 'RHO',
        'SIGMA', 'TAU', 'UPSILON', 'PHI', 'CHI', 'PSI', 'OMEGA'
    ]
    latin_names_lower = [name.lower() for name in latin_names_upper]
    
    # Add all pairs
    safe_print("Creating composite mappings...")
    
    
    # 1. Latin uppercase to uppercase prime
    for letter in latin_upper:
        composite_mapping.add_pair(letter, f"{letter}_prime")
    
    # 2. Latin lowercase to lowercase prime
    for letter in latin_lower:
        composite_mapping.add_pair(letter, f"{letter}_prime")
    
    # 3. Greek uppercase to uppercase prime
    for letter in greek_upper:
        composite_mapping.add_pair(letter, f"{letter}_prime")
    
    # 4. Greek lowercase to lowercase prime
    for letter in greek_lower:
        composite_mapping.add_pair(letter, f"{letter}_prime")
    
    # 5. Latin names uppercase to uppercase prime
    for name in latin_names_upper:
        composite_mapping.add_pair(name, f"{name}_prime")
    
    # 6. Latin names lowercase to lowercase prime
    for name in latin_names_lower:
        composite_mapping.add_pair(name, f"{name}_prime")

    return composite_mapping
    
composite_mapping=create_mapping()



Creating composite mappings...


In [3]:
def inverse_composite(sequence):
    inverse_composite=sequence[::-1]
    for i in range(len(inverse_composite)):
        inverse_composite[i]=composite_mapping.get_element(inverse_composite[i]).composite()
    return inverse_composite


In [4]:
print_with_prime_coloring(inverse_composite(['A_prime','B','C_prime','gamma']))

In [5]:

def detect_forward_sites(DNA,site):
    site_length = len(site)
    start_positions = []
    end_positions = []
    
    # Slide through the DNA looking for matches
    for i in range(len(DNA) - site_length + 1):
        # Check if current window matches the site
        if DNA[i:i+site_length] == site:
            start_positions.append(i)#computer positions
    
    return start_positions

def detect_backward_sites(DNA,site):
    site_length = len(site)
    start_positions = []
    end_positions = []
    
    # Slide through the DNA looking for matches
    for i in range(len(DNA) - site_length + 1):
        # Check if current window matches the site
        if DNA[i:i+site_length] == site[::-1]:
            start_positions.append(i)#computer positions
    
    return start_positions

#ALL positions recorded in natural counting, not computer indices, delete and inverts [site_1,site_2] inclusive
def delete_between(DNA,site_1,site_2):
    return None #DNA[:site_1-1]+DNA[site_2:]  

def invert_between(DNA,site_1,site_2):
    return DNA[:site_1-1]+inverse_composite(DNA[site_1-1:site_2])+DNA[site_2:] 



def evaluate_colour_CRE(DNA,choice1,choice2):

    num1, direction_1 = choice1
    num2, direction_2 = choice2
    #WLOG,make num 1 the smaller one
    if num1 > num2:
        num1, num2 = num2, num1
    
    # Determine which function to use
    #Not yet complete,must know if <> is a flip
    if direction_1 == 'forward':
        if direction_1 == direction_2:
            result= 'DoNotCount'
        elif direction_1!=direction_2:
            result= invert_between(DNA,num1+1,num2+1)
    elif direction_1=='backward':
        if direction_1 == direction_2:
            result= 'DoNotCount'
        else :
            result= 'DoNotCount'

    return result








In [6]:
def probability_calculation(DNA):
    # Dictionary to store counts of each outcome
    outcome_list=[{tuple(DNA):1}]

    for _ in range(10):
        outcome_list.append({})
    recombinases_event=  0


    # Run simulations
    for stage in outcome_list:
        if recombinases_event<len(outcome_list)-1:
            recombinases_event+=1      
            for stored_result,probabilities in stage.items():
                stored_result=list(stored_result)
                num_arrows=0
                forward_9_positions=detect_forward_sites(stored_result,['9'])
                backward_6_positions=detect_backward_sites(stored_result,['6'])
                # Randomly select two sites
                forward = [(num, 'forward') for num in forward_9_positions] 
                backward= [(num, 'backward') for num in backward_6_positions]
                if 1==1:#wierd logic due to historical reasons
                    
                    num_arrows=len(forward)*len(backward)
                    
                    for i in forward:
                        for j in backward:
                            choice1,choice2 = i, j
                            if choice1[0]>choice2[0]:
                                break
                            
                            output = evaluate_colour_CRE(stored_result,choice1,choice2)
                            #while output=='DoNotCount':
                            #    output=evaluate_colour_CRE(stored_result,choice1,choice2)
                            if output!='DoNotCount':
                                formatted_output = tuple(output)
                                #print(formatted_output)#!!!Remove before flight  
                                if formatted_output not in outcome_list[recombinases_event]:
                                    outcome_list[recombinases_event][formatted_output] = 0
                                outcome_list[recombinases_event][formatted_output] +=2*probabilities/num_arrows
                        
        print('stage',recombinases_event,'completed')
    return outcome_list

In [7]:
data={}

outcome_lists= probability_calculation(initial_dna_sequence)


AttributeError: 'NoneType' object has no attribute 'composite'

In [35]:

stored_data=(outcome_lists)

In [36]:
import matplotlib.pyplot as plt
import numpy as np





In [ ]:
x=[]
y=[]
for i in range(len(outcome_lists)):
    x.append(i)
    if i==0:
        y.append(len(outcome_lists[i]))
    elif i >0:
        y.append(len(outcome_lists[i]))


# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot the line graph with log scale on the x-axis
plt.semilogy(x, y, marker='o', linestyle='-', color='b', label='Log X Scale')

# (Alternatively, for log y-axis: plt.semilogy(x, y, ...))
# (For both axes log: plt.loglog(x, y, ...))

# Add labels and title
plt.xlabel('Num of recombinases')
plt.ylabel('Num of possible combinations (log scale)')

plt.legend()
plt.grid(True, which="both", ls="--")
plt.title("Each Stage")

plt.show()

In [38]:
import math
def calculate_entropy(prob_dict):
    """
    Calculate Shannon entropy (in bits) for a dictionary of outcome probabilities.
    
    Args:
        prob_dict (dict): Dictionary where keys are outcomes and values are probabilities.
    
    Returns:
        float: Entropy in bits.
    """
    entropy = 0.0
    for prob in prob_dict.values():
        if prob > 0:  # Avoid log(0) which is undefined
            entropy -= prob * math.log2(prob)
    return entropy

In [ ]:
entropy_list=[]
for _ in outcome_lists:
    entropy_list.append(calculate_entropy(_))
print(entropy_list)
    
x=[]
y=[]
for i in range(len(entropy_list)):
    x.append(i)
    y.append(entropy_list[i])


# Create a figure and axis
plt.figure(figsize=(8, 6))

# Plot the line graph with log scale on the x-axis
plt.plot(x, y, marker='o', linestyle='-', color='b', label='Log X Scale')

# (Alternatively, for log y-axis: plt.semilogy(x, y, ...))
# (For both axes log: plt.loglog(x, y, ...))

# Add labels and title
plt.xlabel('Num of recombinases')
plt.ylabel('Entropy of each stage')
plt.title
plt.legend()
plt.grid(True, which="both", ls="--")

plt.show()